In [4]:
%matplotlib inline
from wordcloud import WordCloud,STOPWORDS
import re
import matplotlib
#import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import nltk
#nltk.download('stopwords')
#nltk.download('words')
stop_words = set(stopwords.words('english'))
#from sklearn.svm import LinearSVC
#from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
#import seaborn as sns
import pyodbc

class OnlinePipeline(Pipeline):
    def partial_fit(self, X, y=None):
        for i, step in enumerate(self.steps):
            name, est = step
            est.partial_fit(X, y)
            if i < len(self.steps) - 1:
               X = est.transform(X)
        return self

def wordcloud_draw(data, color = 'black'):
    words = ' '.join(data)
    wordcloud = WordCloud(stopwords=STOPWORDS,
                      background_color=color,
                      width=2500,
                      height=2000
                     ).generate(words)
    plt.figure(1,figsize=(13, 13))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()

In [5]:
#Method to clean the text of data
def clean_text(text):
    stemmer = SnowballStemmer("english")
    words = set(nltk.corpus.words.words())
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "can not ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = re.sub("\d+", "", text)
    text = text.strip(' ')
    #text = " ".join(w for w in nltk.wordpunct_tokenize(text) \
    #     if w.lower() in words or not w.isalpha())
    text = " ".join(stemmer.stem(w) for w in nltk.wordpunct_tokenize(text))
    #print(text)
    return text

In [8]:
ds_train = [
    ('Charisma East London Phones not working', 'SIM'),
    ('My SIM is not working', 'SIM'),
    ('I have a problem connecting VPN on 3G', 'VPN'),
    ('My Access connection is not working.', 'Access'),
    ('Router for my access connection broken','Access'),
    ('I want update on my SR', 'Update'),
    ('please escalate my SR', 'Escalate'),
    ('Please treat this as escalation','Escalate'),
    ('we have had an escalation','Escalate'),
    ('My phone is not connecting to internet', 'SIM'),
    ('Service Unavailable - Telephony Setup', 'SIM'),
    ('Link down 65675  || AF : Malawi : Blantyre (Head Office) - Puma Office || Inc # 988326', 'VPN'),
    ('Link down', 'VPN'),
    ('connection is horrible', 'VPN')
]
ds_test = [
    ('Phone not working', 'SIM'),
    ('link not working', 'VPN'),
    ('escalation', 'Escalate'),
    ('I want an update for my request', 'Update'),
    ('phone internet issue', 'SIM'),
    ('Access Router offline :(', 'Access')
]

In [9]:
train=pd.DataFrame(ds_train)
test = pd.DataFrame(ds_test)
train.columns = ["Subject","Type"]
test.columns = ["Subject","Type"]

train['Subject'] = train['Subject'].map(lambda com : clean_text(com))
test['Subject'] = test['Subject'].map(lambda com:clean_text(com))
categories = train.Type.values
X_train = train.Subject
X_test = test.Subject

charisma east london phone not work
my sim is not work
i have a problem connect vpn on g
my access connect is not work
router for my access connect broken
i want updat on my sr
pleas escal my sr
pleas treat this as escal
we have had an escal
my phone is not connect to internet
servic unavail telephoni setup
link down af malawi blantyr head offic puma offic inc
link down
connect is horribl
phone not work
link not work
escal
i want an updat for my request
phone internet issu
access router offlin


In [10]:
wordcloud_draw(train.Subject,'black')

NameError: name 'plt' is not defined

In [12]:
# Define a pipeline combining a text feature extractor with multi lable classifier
NB_pipeline = OnlinePipeline([
                ('tfidf', TfidfVectorizer(stop_words=stop_words)),
                ('clf', OneVsRestClassifier(MultinomialNB(
                    fit_prior=True, class_prior=None))),
            ])
NB_pipeline.fit(X_train, train.Type)
prediction = NB_pipeline.predict(X_test)
print('Test accuracy is {}'.format(accuracy_score(test.Type, prediction)))

Test accuracy is 1.0


In [13]:
prediction

array(['SIM', 'VPN', 'Escalate', 'Update', 'SIM', 'Access'], dtype='<U8')

In [14]:
# Classify some text
print(NB_pipeline.predict([clean_text("Phone not working")]))
print(NB_pipeline.predict([clean_text("Link Down")]))
print(NB_pipeline.predict([clean_text("Link fluctating")]))
print(NB_pipeline.predict([clean_text("please escalate this")]))

phone not work
['SIM']
link down
['VPN']
link fluctat
['VPN']
pleas escal this
['Escalate']


In [15]:
print(NB_pipeline.predict([clean_text("Phone Link Down")]))

phone link down
['VPN']


In [16]:
new_test = pd.DataFrame([("Phone Link Down","SIM")])
new_test.columns = ["Subject","Type"]
#Not working -> NB_pipeline.partial_fit(new_test.Subject, new_test.Type)

In [1]:
import pandas as pd
import pyodbc
conn = pyodbc.connect(
    r'DRIVER={FreeTDS};SERVER=zaiss8ouidb3;PORT=1433;DATABASE=siebeldb;UID=DATAREADER;PWD=938884459ee1dffe5c4d5db5ea9a4be4;\
    Encrypt=yes;TrustServerCertificate=yes;Connection Timeout=30;')
sql = "SELECT	SR.SR_NUM, \
		SR.DESC_TEXT AS EMAIL_BODY, \
		SR.SR_TITLE AS EMAIL_SUBJECT, \
		ACC.NAME AS GROUP_NAME \
FROM	S_SRV_REQ SR \
		INNER JOIN S_ORG_EXT ACC ON ACC.ROW_ID = SR.OWNER_OU_ID \
WHERE	SR.CREATED >= GETUTCDATE() - 4 \
		AND SR.SR_SUBTYPE_CD NOT IN ('Monitor')"
ds = pd.read_sql(sql, conn)

In [3]:
ds.head()

,SR_NUM,EMAIL_BODY,EMAIL_SUBJECT,GROUP_NAME
0,1-16402079250,Hi IS\n\nPlease investigate line down - MRP032...,ICM119777220 - MRP0322 - Mrp Ref.: 212061 Pri...,SOC NOC and Telco Assurance
1,1-16402079308,"Hi Team,\r\n\r\nCan you please allocate us 2 d...",RE: Metro Management IP for JHB and ec1 riing...,CoreNOC
2,1-16402079568,Alert: Network Device is Not Responding\nDevic...,ETASK [Neopak] - Networks - Routers: 172.20.55...,SOC NOC and Telco Assurance
3,1-16402079716,Free disk space is less than 20% on volume /va...,Free disk space is less than 20% on volume /va...,Cloud Security Infrastructure
4,1-16402082950,+++ Ilan\r\n\r\nFrom: Sandeep Makan (IS)\r\nSe...,RE: PRP/Sitwala | APN migration,GSC Call Takers


In [ ]:
train=ds[["EMAIL_BODY", "GROUP_NAME"]]
train.columns = ["Subject","Type"]
train['Subject'] = train['Subject'].map(lambda com : clean_text(com))
train.head()

hi is pleas investig line down mrp no log client detail onsit contact person name conroy onsit contact person telephon no altern onsit contact person name lorrain altern onsit contact person telephon no line tag name or alia mrp site name upington site physic address shop a ancorley build market street upington offic hour m f sa su prioriti of call failur descript primari line remain down after flt was conduct mrp ref no is fibr yes circuit number first line troubleshoot flt pleas fill in the relev inform were there power relat issu when incid occur no ping test result sent to is devic power feed sourc is work yes trace rout result sent to is cabl check connector proper seat yes sourc ip address power reset done on the relev equip yes destin ip address log process no s shot of log attach to bodi of call no light status cisco router light status power on sys ge on act ge on poe ge rps conn serial ps telkom diginet devic light status ntu power line line pma dma err ra la telkom eee devic

good day internet solut a dimens data divis attach pleas find your statement of openserv figur number o for with respect to openserv oper invoic kind acknowledg the receipt of this mail to pieterk openserv co za further kind submit remitt advic for payment made to openserv a divis of telkom sa soc ltd to the email address list below in order to facilit accur and timeous alloc for payment receiv pieterk openserv co za last kind notifi openserv a divis of telkom sa soc ltd of ani chang in the contact detail for these statement of openserv figur use email address nation bill oper at pieterk openserv co za kind regard nation bill oper this e mail is subject to the telkom sa soc ltd electron communic legal notic avail at http www openserv co za open media download openserveemaillegalnotic pdf
to iskztvl sent nov id pleas note travel expens no has been approv travel expens local for kubeka toni detail name on passport id toni kubeka is busi unit optinet network addit email flight descript o 

good day team pleas action my request for the follow site excluisv book walmer park walmer park shop centr walmer port elizabeth south africa mb regard sean silverman
hi pleas grant me access to imola idc rogan scholtz thank rogan scholtz system engin tel cell e mail rogan tradebridg co za peterv healthbridg co za web www healthbridg co za https www facebook com healthbridgeza https twitter com healthbridgeza https www linkedin com compani beta a member of tradebridg
hi we would like to regist the follow domain s and point them to oefbsss net dashboard oefbsss net server account name spider black onlin thank sent by eric musandiwa mulovhedzi www stbn co za halfway garden offic park midrand
topup id not go to help support pleas provid usernam of the account api call cancelqueuedtopup allow for the cancel of a queu top up regard tian from db de villier db itexpert co za date wednesday novemb at to tian holm is tian holm is co za is lte support lte support is co za subject top up not load

hello internet solut pleas can you urgent investig the follow line tag numetro nmc hydepark rba adsl mi lydon oliv specialist it team leader descript descript descript descript c user jayw appdata roam microsoft signatur new signiture_fil imag jpg t e mailto lydon numetro co za lydon numetro co za address hutton court st floor summit road cnr jan smut dunkeld west http numetro co za numetro co za
hi team pleas activ the below sim number cost centr rhf user name thobeka koman regard pam qina l intern technolog architectur l group it l group servic tel l qinapa aforb co za alexand forb group servic pti ltd l l po box sandton south africa l west street sandown co reg no l www alexanderforb co za http www alexanderforb co za af logo jpg tree png pleas consid your environment respons befor print this e mail report fraud and uneth behaviour in alexand forb by call our toll free whistl blower line on south africa and namibia onli or visit www whistleblow co za http www whistleblow co za _____

hi domain sar would like to transfer the domain below to vodacom as thet won the tower i tender could you pleas let me know what the process is to releas the domain and give auth code so that sar can transfer them pleas let me know whether you requir anyth els kind regard ernest sent from my samsung galaxi smartphon origin messag from johann klopper jklopper sar gov za date gmt to ernest van vuuren is e is co za cc adri rademey is adri rademey is co za subject re sar dns zone file ernest i requir author code to transfer below dns record to vodacom can you suppli thank regard johann klopper manag network servic sar it cid imag jpg dee mobil cid imag jpg dee phone cid imag jpg dee fax cid imag jpg dee e mail jklopper sar gov za mailto jklopper sar gov za floor block f sar head offic la hae la sar pretoria south africa from ernest van vuuren is mailto e is co za sent juli pm to johann klopper cc adri rademey is subject sar dns zone file hi johann as request below are all the domain that i

...

good day team pleas action my request for the follow site maxwel street empangeni maxwel street mb smith street escov hous smith street mb th avenu walmer shop g sixth avenu shop centr cnr th heugh road mb aberdeen cnr hoop voortrekk street mb adderley street adderley street mb adelaid cnr welsh church street mb albert street shop st floor qualbert centr bertha mkhize street mb albertinia main street mb alexandra alexandra plaza rd street mb alexandria cnr voortrekk krogh street mb aliw north cnr somerset collin street mb athlon cnr old klipfontein road th avenu mb atlasvill shop atlas mall cnr finch reier road mb ba phalaborwa shop eden squar cnr nelson mandela palm avenu mb barberton shop jock of the bushveld shop centr cnr van niekerk general street mb bark east cole street mb barrydal van riebeeck street mb beaufort west donkin street mb bela bela bosveld pick n pay centr cnr mark potgiet street mb belfast b fitzgerald street mb belhar shop airport shop centr cnr stellenbosch arter

good day team pleas action my request for the follow site maxwel street empangeni maxwel street mb smith street escov hous smith street mb th avenu walmer shop g sixth avenu shop centr cnr th heugh road mb aberdeen cnr hoop voortrekk street mb adderley street adderley street mb adelaid cnr welsh church street mb albert street shop st floor qualbert centr bertha mkhize street mb albertinia main street mb alexandra alexandra plaza rd street mb alexandria cnr voortrekk krogh street mb aliw north cnr somerset collin street mb athlon cnr old klipfontein road th avenu mb atlasvill shop atlas mall cnr finch reier road mb ba phalaborwa shop eden squar cnr nelson mandela palm avenu mb barberton shop jock of the bushveld shop centr cnr van niekerk general street mb bark east cole street mb barrydal van riebeeck street mb beaufort west donkin street mb bela bela bosveld pick n pay centr cnr mark potgiet street mb belfast b fitzgerald street mb belhar shop airport shop centr cnr stellenbosch arter

good day team pleas action my request for the follow site maxwel street empangeni maxwel street mb smith street escov hous smith street mb th avenu walmer shop g sixth avenu shop centr cnr th heugh road mb aberdeen cnr hoop voortrekk street mb adderley street adderley street mb adelaid cnr welsh church street mb albert street shop st floor qualbert centr bertha mkhize street mb albertinia main street mb alexandra alexandra plaza rd street mb alexandria cnr voortrekk krogh street mb aliw north cnr somerset collin street mb athlon cnr old klipfontein road th avenu mb atlasvill shop atlas mall cnr finch reier road mb ba phalaborwa shop eden squar cnr nelson mandela palm avenu mb barberton shop jock of the bushveld shop centr cnr van niekerk general street mb bark east cole street mb barrydal van riebeeck street mb beaufort west donkin street mb bela bela bosveld pick n pay centr cnr mark potgiet street mb belfast b fitzgerald street mb belhar shop airport shop centr cnr stellenbosch arter

good day team pleas action my request for the follow site maxwel street empangeni maxwel street mb smith street escov hous smith street mb th avenu walmer shop g sixth avenu shop centr cnr th heugh road mb aberdeen cnr hoop voortrekk street mb adderley street adderley street mb adelaid cnr welsh church street mb albert street shop st floor qualbert centr bertha mkhize street mb albertinia main street mb alexandra alexandra plaza rd street mb alexandria cnr voortrekk krogh street mb aliw north cnr somerset collin street mb athlon cnr old klipfontein road th avenu mb atlasvill shop atlas mall cnr finch reier road mb ba phalaborwa shop eden squar cnr nelson mandela palm avenu mb barberton shop jock of the bushveld shop centr cnr van niekerk general street mb bark east cole street mb barrydal van riebeeck street mb beaufort west donkin street mb bela bela bosveld pick n pay centr cnr mark potgiet street mb belfast b fitzgerald street mb belhar shop airport shop centr cnr stellenbosch arter

good day team pleas action my request for the follow site maxwel street empangeni maxwel street mb smith street escov hous smith street mb th avenu walmer shop g sixth avenu shop centr cnr th heugh road mb aberdeen cnr hoop voortrekk street mb adderley street adderley street mb adelaid cnr welsh church street mb albert street shop st floor qualbert centr bertha mkhize street mb albertinia main street mb alexandra alexandra plaza rd street mb alexandria cnr voortrekk krogh street mb aliw north cnr somerset collin street mb athlon cnr old klipfontein road th avenu mb atlasvill shop atlas mall cnr finch reier road mb ba phalaborwa shop eden squar cnr nelson mandela palm avenu mb barberton shop jock of the bushveld shop centr cnr van niekerk general street mb bark east cole street mb barrydal van riebeeck street mb beaufort west donkin street mb bela bela bosveld pick n pay centr cnr mark potgiet street mb belfast b fitzgerald street mb belhar shop airport shop centr cnr stellenbosch arter

In [ ]:
test = pd.DataFrame(ds_test)

test.columns = ["Subject","Type"]


test['Subject'] = test['Subject'].map(lambda com:clean_text(com))
categories = train.Type.values
X_train = train.Subject
X_test = test.Subject